# Tutorial 1: Introduction to ASCICat

## Activity-Stability-Cost Integrated Catalyst Discovery

This tutorial introduces the ASCICat framework for multi-objective electrocatalyst screening. You will learn:

1. The scientific foundation behind ASCI scoring
2. How to install and set up ASCICat
3. A complete quick-start workflow
4. Understanding the output metrics

---

## 1. Scientific Background

### The Challenge in Electrocatalysis

Electrocatalyst discovery involves optimizing multiple competing objectives:

- **Activity**: How effectively does the catalyst drive the reaction? (Sabatier principle)
- **Stability**: How resistant is the catalyst to electrochemical degradation?
- **Cost**: Is the catalyst economically viable for large-scale deployment?

Traditional approaches often optimize for a single metric, missing the holistic picture needed for practical applications.

### The ASCI Solution

ASCICat provides a unified framework that combines all three criteria into a single, transparent metric:

$$\phi_{ASCI} = w_a \cdot S_a(\Delta E) + w_s \cdot S_s(\gamma) + w_c \cdot S_c(C)$$

Where:
- $S_a$ = Activity score based on adsorption energy
- $S_s$ = Stability score based on surface energy
- $S_c$ = Cost score based on material price
- $w_a, w_s, w_c$ = User-defined weights (sum to 1)

## 2. Installation and Setup

First, ensure ASCICat is installed:

In [ ]:
# If running from the repository, add the parent directory to path
import sys
sys.path.insert(0, '..')

# Import the main ASCICat components
from ascicat import ASCICalculator
from ascicat.visualizer import Visualizer

# Standard scientific libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

print("ASCICat imported successfully!")

## 3. Quick Start: Complete Workflow

Let's run through a complete catalyst screening workflow in just a few steps.

### Step 1: Initialize the Calculator

Create an ASCICalculator for the Hydrogen Evolution Reaction (HER):

In [ ]:
# Initialize calculator for HER
calc = ASCICalculator(
    reaction='HER',           # Reaction type: 'HER' or 'CO2RR'
    scoring_method='linear'   # Scoring method: 'linear' or 'gaussian'
)

print(f"Reaction: {calc.reaction}")
print(f"Optimal adsorption energy: {calc.config.optimal_energy} eV")
print(f"Activity width (sigma): {calc.config.activity_width} eV")

### Step 2: Load Catalyst Data

Load the HER catalyst dataset containing DFT-computed properties:

In [ ]:
# Load the HER catalyst data
calc.load_data('../data/HER_clean.csv')

# Preview the data
print(f"\nLoaded {len(calc.data)} catalysts")
print("\nData columns:")
print(calc.data.columns.tolist())

# Show first few rows
calc.data.head()

### Step 3: Calculate ASCI Scores

Compute ASCI scores with equal weights for activity, stability, and cost:

In [ ]:
# Calculate ASCI with equal weights
results = calc.calculate_asci(
    w_a=0.33,  # Activity weight
    w_s=0.33,  # Stability weight
    w_c=0.34   # Cost weight
)

print(f"\nCalculated ASCI scores for {len(results)} catalysts")
print("\nNew columns added:")
print(['activity_score', 'stability_score', 'cost_score', 'ASCI', 'rank'])

### Step 4: Explore Top Catalysts

Retrieve and display the top-performing catalysts:

In [ ]:
# Get top 10 catalysts
top_10 = calc.get_top_catalysts(n=10)

# Display key metrics
display_cols = ['symbol', 'DFT_ads_E', 'surface_energy', 'Cost', 
                'activity_score', 'stability_score', 'cost_score', 'ASCI', 'rank']

# Filter to available columns
available_cols = [c for c in display_cols if c in top_10.columns]
top_10[available_cols].round(3)

### Step 5: Visualize Results

Generate a quick visualization of the ASCI score distribution:

In [ ]:
# Create a simple visualization
fig, axes = plt.subplots(1, 3, figsize=(14, 4))

# Panel 1: ASCI distribution
axes[0].hist(results['ASCI'], bins=30, color='steelblue', edgecolor='white', alpha=0.8)
axes[0].axvline(results['ASCI'].mean(), color='red', linestyle='--', label=f'Mean: {results["ASCI"].mean():.3f}')
axes[0].set_xlabel('ASCI Score')
axes[0].set_ylabel('Count')
axes[0].set_title('ASCI Score Distribution')
axes[0].legend()

# Panel 2: ASCI vs Adsorption Energy
scatter = axes[1].scatter(results['DFT_ads_E'], results['ASCI'], 
                          c=results['ASCI'], cmap='viridis', alpha=0.6, s=20)
axes[1].axvline(calc.config.optimal_energy, color='red', linestyle='--', 
                label=f'Optimal: {calc.config.optimal_energy} eV')
axes[1].set_xlabel('Adsorption Energy (eV)')
axes[1].set_ylabel('ASCI Score')
axes[1].set_title('ASCI vs Adsorption Energy')
axes[1].legend()

# Panel 3: Component scores for top 10
top_10_plot = top_10.head(10).copy()
x = np.arange(len(top_10_plot))
width = 0.25

axes[2].bar(x - width, top_10_plot['activity_score'], width, label='Activity', color='#2ecc71')
axes[2].bar(x, top_10_plot['stability_score'], width, label='Stability', color='#3498db')
axes[2].bar(x + width, top_10_plot['cost_score'], width, label='Cost', color='#e74c3c')
axes[2].set_xlabel('Catalyst Rank')
axes[2].set_ylabel('Component Score')
axes[2].set_title('Top 10: Component Breakdown')
axes[2].set_xticks(x)
axes[2].set_xticklabels([f'#{i+1}' for i in range(len(top_10_plot))])
axes[2].legend(loc='lower right')

plt.tight_layout()
plt.show()

## 4. Understanding the Output

### Score Interpretation

All scores are normalized to the range [0, 1]:

| Score | Range | Interpretation |
|-------|-------|----------------|
| Activity | 0-1 | Higher = closer to Sabatier optimum |
| Stability | 0-1 | Higher = lower surface energy |
| Cost | 0-1 | Higher = cheaper material |
| ASCI | 0-1 | Higher = better overall candidate |

In [ ]:
# Summary statistics
print("Score Statistics:")
print("="*50)
for col in ['activity_score', 'stability_score', 'cost_score', 'ASCI']:
    print(f"{col:20s}: mean={results[col].mean():.3f}, std={results[col].std():.3f}, max={results[col].max():.3f}")

### Saving Results

Export results for further analysis:

In [ ]:
# Save to CSV (uncomment to run)
# results.to_csv('my_asci_results.csv', index=False)
# print("Results saved to my_asci_results.csv")

# Or save just the top catalysts
# top_10.to_csv('my_top_catalysts.csv', index=False)

## 5. Next Steps

Now that you understand the basics, explore more advanced features:

- **Tutorial 2**: Complete HER screening workflow
- **Tutorial 3**: CO2RR multi-pathway analysis
- **Tutorial 4**: Advanced visualization techniques
- **Tutorial 5**: Sensitivity analysis and weight optimization

---

**Congratulations!** You've completed your first ASCICat workflow.